In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec, cm
FONT_SIZE = 17

from random import shuffle, seed
SEED_NUMBER=42

from pathlib import Path

In [2]:
proj_path = Path().resolve().parent
print(proj_path)

/home/mommess/Documents/Vectorpack/experiments_vector_paper/results_data


In [3]:
file_DI = proj_path / "BPPLIB_Difficult_Instances_dim1.csv"
file_F = proj_path / "BPPLIB_Falkenauer_dim1_OPT.csv"
file_H28 = proj_path / "BPPLIB_Hard28_dim1_OPT.csv"
file_I = proj_path / "BPPLIB_Irnich_dim1_OPT.csv"
file_S = proj_path / "BPPLIB_Scholl_dim1_OPT.csv"
file_Sw = proj_path / "BPPLIB_Schwerin_dim1_OPT.csv"
file_W = proj_path / "BPPLIB_Waescher_dim1_OPT.csv"

In [4]:
def load_df(filename):
    df = pd.read_csv(filename, sep='\t')
    
    df.rename(columns={"WFDm":"MB-WFD", "WFDm_timems":"MB-WFD_timems"}, inplace=True)
    
    # Keep only 1D algos (WF removed because significantly bad)
    alg_names = ["FF", "BF", "FFD", "BFD", "WFD", "MB-WFD"]
    alg_names_time = [x+"_timems" for x in alg_names]
    all_cols = ["instance_name", "LB"] +alg_names + alg_names_time
    if "OPT" in df.columns:
        all_cols.insert(2, "OPT")
    df = df[all_cols]
            
    # Replace '-1' values by the FF solutions in MB-WFD
    df.loc[df["MB-WFD"] == -1, "MB-WFD"] = df['FF']
    
    if "OPT" in df.columns:
        df.rename(columns={"OPT":"LB_or_OPT"}, inplace=True)
    else:
        df.insert(2, 'LB_or_OPT', df["LB"])
        if df["LB"].min() <= 0: # Just in case
            print("ERROR LB = 0 sometimes for", filename)
    
    return df, list(alg_names)

In [5]:
def get_results(df, alg_names):
    best_sol = df[alg_names].min(axis=1)

    time_cols = [i+'_timems' for i in alg_names]
    
    eps = df[alg_names].apply(lambda x: (round(x / df["LB_or_OPT"] - 1, 3))*100) # *100 to get a percentage value
    time = df[time_cols].apply(lambda x: round(x, 4))
    time.columns = alg_names
    diff = df[alg_names].apply(lambda x: x - df["LB_or_OPT"])
    #diff_best = df[alg_names].apply(lambda x: x - best_sol)
    
    #match_LB = df[alg_names].apply(lambda x: x == df["LB"])
    #best = df[alg_names].apply(lambda x: x == best_sol)

    return eps, time, diff#, diff_best, best, match_LB

In [6]:
def get_results_by(df, alg_names, by_list, group_as_index=False):
    sub_df = df[by_list].copy()
    
    best_sol = df[alg_names].min(axis=1)

    time_cols = [i+'_timems' for i in alg_names]
    
    eps = df[alg_names].apply(lambda x: (round(x / df["LB_or_OPT"] - 1, 3))*100) # *100 to get a percentage value
    time = df[time_cols].apply(lambda x: round(x, 4))
    time.columns = alg_names
    diff = df[alg_names].apply(lambda x: x - df["LB_or_OPT"])
    #diff_best = df[alg_names].apply(lambda x: x - best_sol)
    
    #match_LB = df[alg_names].apply(lambda x: x == df["LB"])
    #best = df[alg_names].apply(lambda x: x == best_sol)
    
    eps = pd.concat([sub_df, eps], axis=1).groupby(by=by_list, as_index=group_as_index).mean()
    time = pd.concat([sub_df, time], axis=1).groupby(by=by_list, as_index=group_as_index).mean()
    diff = pd.concat([sub_df, diff], axis=1).groupby(by=by_list, as_index=group_as_index).mean()
    #diff_best = pd.concat([sub_df, diff_best], axis=1).groupby(by=by_list, as_index=group_as_index).mean()
    
    #best = pd.concat([sub_df, best], axis=1).groupby(by=by_list, as_index=group_as_index).sum()
    #match_LB = pd.concat([sub_df, match_LB], axis=1).groupby(by=by_list, as_index=group_as_index).sum()
    
    return eps, time, diff#, diff_best, best, match_LB

## Load datasets and get results

In [7]:
df_DI, alg_names = load_df(file_DI)

df_F, _ = load_df(file_F)
df_H28, _ = load_df(file_H28)
df_S, _ = load_df(file_S)
df_Sw, _ = load_df(file_Sw)
df_W, _ = load_df(file_W)

df_I, _ = load_df(file_I)

In [9]:
#results_DI = get_results(df_DI, alg_names) # has 2 classes
#results_F = get_results(df_F, alg_names) # has classes
results_H28 = get_results(df_H28, alg_names)
#results_S = get_results(df_S, alg_names) # has classes
#results_Sw = get_results(df_Sw, alg_names) # has classes
results_W = get_results(df_W, alg_names)

results_I = get_results(df_I, alg_names)

In [10]:
# 2 classes of difficult instances
df_DI["class"] = df_DI["instance_name"].apply(lambda x: x.split('_')[2])
results_ANI = get_results(df_DI[df_DI["class"] == "NR"], alg_names)
results_AI = get_results(df_DI[df_DI["class"] == "DI"], alg_names)

In [11]:
# Falkenauer has 2 classes
df_F["class"] = df_F["instance_name"].apply(lambda x: x.split('_')[1][0].upper())
results_FT = get_results(df_F[df_F["class"] == "T"], alg_names)
results_FU = get_results(df_F[df_F["class"] == "U"], alg_names)

In [12]:
def get_Scholl_class(x):
    if x[2] == "C":
        return "1"
    elif x[2] == "W":
        return "2"
    elif x[2] == "R":
        return "3"
    else:
        return "0"

In [13]:
# Scholl has 3 classes
df_S["class"] = df_S["instance_name"].apply(lambda x: get_Scholl_class(x))
results_S1 = get_results(df_S[df_S["class"] == "1"], alg_names)
results_S2 = get_results(df_S[df_S["class"] == "2"], alg_names)
results_S3 = get_results(df_S[df_S["class"] == "3"], alg_names)

In [14]:
# Schwerwin has 2 classes
df_Sw["class"] = df_Sw["instance_name"].apply(lambda x: x[8])
results_Sw1 = get_results(df_Sw[df_Sw["class"] == "1"], alg_names)
results_Sw2 = get_results(df_Sw[df_Sw["class"] == "2"], alg_names)

In [15]:
dict_res = {
    "Difficult ANI": (results_ANI, "ANI"),
    "Difficult AI": (results_AI, "AI"),
    "Falkenauer Triplets": (results_FT, "FT"),
    "Falkenauer Uniform": (results_FU, "FU"),
    "Hard 28": (results_H28, "H28"),
    "Irnich": (results_I, "I"),
    "Scholl class 1": (results_S1, "S1"),
    "Scholl class 2": (results_S2, "S2"),
    "Scholl class 3": (results_S3, "S3"),
    "Schwerin class 1": (results_Sw1, "Sw1"),
    "Schwerin class 2": (results_Sw2, "Sw2"),
    "Waescher": (results_W, "W"),
}

## Check algorithms performing equally

In [121]:
for name, (res, _) in dict_res.items():
    nb_inst = res[0].shape[0]
    print(f"Bench {name} with {nb_inst} instances")
    
    sub_list = alg_names.copy()
    
    for i in range(len(alg_names)):
        algo_test = sub_list.pop(0)
        counts = res[0][sub_list].apply(lambda x: x == res[0][algo_test]).sum()
        ll = list(counts[counts == nb_inst].index)
        if (len(ll) > 0):
            print(f"   {algo_test} identical to {ll}")
            sub_list = [x for x in sub_list if not x in ll]
        if len(sub_list) == 0:
            break
            
    print()

Bench Difficult ANI with 250 instances
   FFD identical to ['BFD', 'MB-WFD']

Bench Difficult AI with 250 instances
   FFD identical to ['BFD', 'MB-WFD']

Bench Falkenauer Triplets with 80 instances
   FF identical to ['BF']
   FFD identical to ['BFD', 'WFD']

Bench Falkenauer Uniform with 80 instances
   BFD identical to ['MB-WFD']

Bench Hard 28 with 28 instances
   FFD identical to ['BFD', 'MB-WFD']

Bench Irnich with 240 instances
   BFD identical to ['MB-WFD']

Bench Scholl class 1 with 720 instances
   BFD identical to ['MB-WFD']

Bench Scholl class 2 with 480 instances
   FFD identical to ['BFD']

Bench Scholl class 3 with 10 instances
   FF identical to ['BF']
   FFD identical to ['BFD', 'WFD', 'MB-WFD']

Bench Schwerin class 1 with 100 instances
   FF identical to ['BF']
   FFD identical to ['BFD', 'WFD', 'MB-WFD']

Bench Schwerin class 2 with 100 instances
   FF identical to ['BF']
   FFD identical to ['BFD', 'WFD', 'MB-WFD']

Bench Waescher with 17 instances
   FF identical 

In [48]:
def my_sup(x):
    return x[x > 0].count()


def my_eq(x):
    return x[x == 0].count()

def my_low(x):
    return x[x < 0].count()

def make_diff(df, list1, list2):
    a = df[list1]
    b = df[list2]
    b.columns = a.columns
    df_diff = (a - b).agg(['mean', 'std', my_sup, my_eq, my_low])
    return df_diff

# When comparing algo A with algo B
# Mean is negative when algo A is better than B
# my_sup counts when sol of A is worse than B
# my_low counts when sol of A is better than B

In [49]:
make_diff(big_df, ['FFD'], ['BFD'])

,FFD
mean,-0.002837
std,0.053199
my_sup,0.000000
my_eq,2109.000000
my_low,6.000000


In [50]:
make_diff(big_df, ['FFD'], ['MB-WFD'])

,FFD
mean,0.120095
std,0.661660
my_sup,109.000000
my_eq,2000.000000
my_low,6.000000


## Plots

In [15]:
def create_small_colors(n):
    cmap = cm.get_cmap('turbo')
    n+=1
    if n > 1:
        c = []
        for i in range(n):
            c.append(cmap(float(i/(n-1))))
        #shuffle(c)
    else:
        c = [cmap(0)]
    return c[1:]

In [16]:
def plot_barchart(df, alg_names, 
                  title, ylabel,
                  fname=None):
    x = np.arange(len(alg_names))
    
    color_list = create_small_colors(len(alg_names))
    
    fig = plt.figure(figsize=(15,10))

    plt.bar(x, df.mean(), yerr=df.std(), color=color_list, capsize=10)
    plt.xticks(x, alg_names, rotation=45, fontsize=FONT_SIZE)
    plt.yticks(fontsize=FONT_SIZE)
    plt.ylabel(ylabel, fontsize=FONT_SIZE+8)
    plt.title(title, fontsize=FONT_SIZE+8)

    plt.show()
    
    if fname:
        fig.savefig(fname, bbox_inches='tight')

### Plot percentage error

In [ ]:
for name, (results, short_name) in dict_res.items():
    fname = proj_path / f"plots/1dim/barplot_{short_name}.pdf"
    plot_barchart(results[0], alg_names,
                  f"{name} instances", "Average % error to LB",
                  fname=fname)

In [66]:
results_W[0].describe()

,FF,BF,FFD,BFD,WFD,MB-WFD
count,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
mean,5.700000,5.700000,5.441176,5.441176,5.441176,5.441176
std,2.739982,2.739982,2.645056,2.645056,2.645056,2.645056
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.200000,4.200000,4.200000,4.200000,4.200000,4.200000
50%,6.200000,6.200000,6.200000,6.200000,6.200000,6.200000
75%,7.700000,7.700000,7.100000,7.100000,7.100000,7.100000
max,9.100000,9.100000,9.100000,9.100000,9.100000,9.100000


### Plot additional bins

In [ ]:
for name, (results, short_name) in dict_res.items():
    fname = proj_path / f"plots/1dim/bins/barplot_bins_{short_name}.pdf"
    plot_barchart(results[2], alg_names,
                  f"{name} instances", "Average additional bins to LB",
                  fname=fname)

### Print stats per benchmark and class

In [136]:
for name, (results, short_name) in dict_res.items():
    print(name)
    for alg in alg_names:
        print(f"\t{alg}")
        print(f"\t\t%error: {results[0][alg].mean():.3f}")
        print(f"\t\ttime {results[1][alg].mean():.2f}")
        print(f"\t\tadditional bins: {results[2][alg].mean():.2f}")

Difficult ANI
	FF
		%error: 3.530
		time 2.16
		additional bins: 6.37
	BF
		%error: 2.501
		time 17.00
		additional bins: 4.33
	FFD
		%error: 0.700
		time 3.43
		additional bins: 1.00
	BFD
		%error: 0.700
		time 13.60
		additional bins: 1.00
	WFD
		%error: 0.904
		time 8.16
		additional bins: 1.52
	MB-WFD
		%error: 0.700
		time 68.90
		additional bins: 1.00
Difficult AI
	FF
		%error: 3.651
		time 2.20
		additional bins: 6.72
	BF
		%error: 2.590
		time 17.13
		additional bins: 4.86
	FFD
		%error: 0.700
		time 3.50
		additional bins: 1.00
	BFD
		%error: 0.700
		time 13.82
		additional bins: 1.00
	WFD
		%error: 0.913
		time 8.32
		additional bins: 1.54
	MB-WFD
		%error: 0.700
		time 68.56
		additional bins: 1.00
Falkenauer Triplets
	FF
		%error: 12.876
		time 0.35
		additional bins: 9.40
	BF
		%error: 12.876
		time 3.46
		additional bins: 9.40
	FFD
		%error: 14.707
		time 0.50
		additional bins: 11.01
	BFD
		%error: 14.707
		time 1.99
		additional bins: 11.01
	WFD
		%error: 14.707
		time 

### Average running times

In [69]:
for name, (results, short_name) in dict_res.items():
    print(f"--- {name} ---")
    print(results[1].mean())
    print("\n")

--- Difficult ANI ---
FF         2.160
BF        17.004
FFD        3.432
BFD       13.600
WFD        8.160
MB-WFD    68.900
dtype: float64


--- Difficult AI ---
FF         2.200
BF        17.128
FFD        3.504
BFD       13.816
WFD        8.320
MB-WFD    68.564
dtype: float64


--- Falkenauer Triplets ---
FF        0.3500
BF        3.4625
FFD       0.5000
BFD       1.9875
WFD       1.0875
MB-WFD    2.9625
dtype: float64


--- Falkenauer Uniform ---
FF         2.4125
BF        16.4125
FFD        2.9875
BFD        9.4750
WFD        6.1250
MB-WFD    63.7750
dtype: float64


--- Hard 28 ---
FF        0.000000
BF        1.428571
FFD       0.035714
BFD       0.964286
WFD       0.250000
MB-WFD    4.714286
dtype: float64


--- Scholl class 1 ---
FF         0.543056
BF         4.604167
FFD        0.697222
BFD        2.905556
WFD        1.869444
MB-WFD    17.533333
dtype: float64


--- Scholl class 2 ---
FF        0.150000
BF        1.595833
FFD       0.252083
BFD       1.041667
WFD       0.53

## Check optimal solutions found

In [70]:
def print_optimal_found(df, alg_names, full=False):
    best_sol = df[alg_names].min(axis=1)
    n = (df["LB_or_OPT"] == best_sol).sum()
    print(f"\t{n} instances proved to optimality out of {df.shape[0]}")
    
    if full:
        dff = df[alg_names].apply(lambda x: x == df["LB_or_OPT"])
        print(dff.sum())

In [71]:
dict_dfs = {
    "Difficult ANI": df_DI[df_DI["class"] == "NR"],
    "Difficult AI": df_DI[df_DI["class"] == "DI"],
    "Falkenauer Triplet": df_F[df_F["class"] == "T"],
    "Falkenauer Uniform": df_F[df_F["class"] == "U"],
    "Hard 28": df_H28,
    "Scholl 1": df_S[df_S["class"] == "1"],
    "Scholl 2": df_S[df_S["class"] == "2"],
    "Scholl 3": df_S[df_S["class"] == "3"],
    "Schwerin 1": df_Sw[df_Sw["class"] == "1"],
    "Schwerin 2": df_Sw[df_Sw["class"] == "2"],
    "Waescher": df_W
}

In [72]:
for name, df in dict_dfs.items():
    print(name)
    print_optimal_found(df, alg_names, False)

Difficult ANI
	0 instances proved to optimality out of 250
Difficult AI
	0 instances proved to optimality out of 250
Falkenauer Triplet
	0 instances proved to optimality out of 80
Falkenauer Uniform
	6 instances proved to optimality out of 80
Hard 28
	5 instances proved to optimality out of 28
Scholl 1
	546 instances proved to optimality out of 720
Scholl 2
	238 instances proved to optimality out of 480
Scholl 3
	0 instances proved to optimality out of 10
Schwerin 1
	0 instances proved to optimality out of 100
Schwerin 2
	0 instances proved to optimality out of 100
Waescher
	2 instances proved to optimality out of 17


## Plot % error as a function of n

In [16]:
def get_scholl_n(i):
    if i == "1":
        return 50
    elif i == "2":
        return 100
    elif i == "3":
        return 200
    elif i == "4":
        return 500

In [17]:
# Quick and dirty: retrive the value of n for each instance
# Falkenauer instances
df_F["n"] = df_F["instance_name"].apply(lambda x: int(x.split('_')[1][1:]))

# Difficult Instances
df_DI["n"] = 0
df_DI.loc[df_DI["class"] == "NR","n"] = df_DI.loc[df_DI["class"] == "NR","instance_name"].apply(lambda x: int(x.split('_')[0]))
df_DI.loc[df_DI["class"] == "DI","n"] = df_DI.loc[df_DI["class"] == "DI","instance_name"].apply(lambda x: 1+int(x.split('_')[0]))

# Scholl
df_S["n"] = 200
df_S.loc[df_S["class"] != "3", "n"] = df_S.loc[df_S["class"] != "3", "instance_name"].apply(lambda x: get_scholl_n(x[1]))

# Schwerin has constant n values of 100 for class 1 and 120 for class 2
#df_Sw["n"] = df_Sw["class"].apply(lambda x: 100 if x == "1" else 120)

# Hard 28, Waescher and Irnich have non regular n values

In [20]:
def plot_n_eps(y_data, alg_list, stitle, xlabel, ylabel, filename=None):
    fig = plt.figure(figsize=(15,8))

    color_list = create_small_colors(len(alg_list))

    i = 0
    legend_markers = []
    legend_names = []
    for alg in alg_list:
        plt.plot(y_data[alg], '.:', c=color_list[i], linewidth=2, markersize=15)
        legend_markers.append(plt.Line2D([0,0],[0,0], color=color_list[i], marker = 'o', linestyle=''))
        legend_names.append(alg)
        i+=1
            
    fig.legend(legend_markers, legend_names, numpoints=1,
               loc='lower center', fontsize=FONT_SIZE,
               ncol=6, bbox_to_anchor=(0.5, -0.04))

    plt.xticks(y_data.index, fontsize=FONT_SIZE)
    plt.yticks(fontsize=FONT_SIZE)
    plt.title(stitle, fontsize=FONT_SIZE+2)
    plt.xlabel(xlabel, fontsize=FONT_SIZE+2)
    plt.ylabel(ylabel, fontsize=FONT_SIZE+2)

    plt.show()
    if filename:
        fig.savefig(filename, bbox_inches='tight')

In [21]:
results_F = get_results_by(df_F, alg_names, ["class", "n"], True)
results_DI = get_results_by(df_DI, alg_names, ["class", "n"], True)
results_S = get_results_by(df_S, alg_names, ["class", "n"], True)

In [ ]:
fname = proj_path / "plots/1dim/lines_Falkenauer_U.pdf"
plot_n_eps(results_F[0].loc['U'], alg_names,
           f"Falkenauer Uniform", "Number of items", "% error",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/lines_Falkenauer_T.pdf"
plot_n_eps(results_F[0].loc['T'], alg_names,
           f"Falkenauer Triplet", "Number of items", "% error",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/lines_Difficult_ANI.pdf"
plot_n_eps(results_DI[0].loc['NR'], alg_names,
           f"Difficult ANI", "Number of items", "% error",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/lines_Difficult_AI.pdf"
plot_n_eps(results_DI[0].loc['DI'], alg_names,
           f"Difficult AI", "Number of items", "% error",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/lines_Scholl1.pdf"
plot_n_eps(results_S[0].loc['1'], alg_names,
           f"Scholl 1", "Number of items", "% error",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/lines_Scholl2.pdf"
plot_n_eps(results_S[0].loc['2'], alg_names,
           f"Scholl 2", "Number of items", "% error",
           filename=fname)

### Plot additional bins as a function of n

In [ ]:
fname = proj_path / "plots/1dim/bins/lines_bins_Falkenauer_U.pdf"
plot_n_eps(results_F[2].loc['U'], alg_names,
           f"Falkenauer Uniform", "Number of items", "Average additional bins to LB",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/bins/lines_bins_Falkenauer_T.pdf"
plot_n_eps(results_F[2].loc['T'], alg_names,
           f"Falkenauer Triplet", "Number of items", "Average additional bins to LB",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/bins/lines_bins_Difficult_ANI.pdf"
plot_n_eps(results_DI[2].loc['NR'], alg_names,
           f"Difficult ANI", "Number of items", "Average additional bins to LB",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/bins/lines_bins_Difficult_AI.pdf"
plot_n_eps(results_DI[2].loc['DI'], alg_names,
           f"Difficult AI", "Number of items", "Average additional bins to LB",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/bins/lines_bins_Scholl1.pdf"
plot_n_eps(results_S[0].loc['1'], alg_names,
           f"Scholl 1", "Number of items", "Average additional bins to LB",
           filename=fname)

In [ ]:
fname = proj_path / "plots/1dim/bins/lines_bins_Scholl2.pdf"
plot_n_eps(results_S[0].loc['2'], alg_names,
           f"Scholl 2", "Number of items", "Average additional bins to LB",
           filename=fname)

### Plot additional bins divided by n, as a function of n

In [ ]:
fname = proj_path / "plots/1dim/bins_lines_bins_Falkenauer_U.pdf"
dd = (results_F[2].loc['U']).divide(results_F[2].loc['U'].index, axis=0)
plot_n_eps(dd, alg_names,
           f"Falkenauer Uniform", "Number of items", "Additional bins per item",
           filename=fname)